In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 17351076
paper_name = 'ohnuki_ohya_2007' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Table2.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 58 x 11


In [7]:
original_data.head()

,No.,Gene name (standard/systematic),Growth typea,Divalent cation sensitivity,Unnamed: 4,Unnamed: 5,Unnamed: 6,Pet phenotype,Quinacrineb,Ca2+content (nmole Ca2+/mg of protein),Classc
0,1,afg3/yer017c,C,Ca,NaN,NaN,NaN,−,Yes,1.41 ± 0.03,II
1,2,bud25/yer014c-a,B,Ca,NaN,NaN,NaN,−,Yes,2.78 ± 0.10,NaN
2,3,bud32/ygr262c,B,Ca,NaN,NaN,NaN,−,Yes,0.418 ± 0.025,NaN
3,4,cls2/ybr036c,A,Ca,NaN,NaN,NaN,+,Yes,1.17 ± 0.03,NaN
4,5,ctr1/ypr124w,A,Ca,NaN,NaN,NaN,−,Yes,0.462 ± 0.043,NaN


In [8]:
original_data['orf'] = original_data['Gene name (standard/systematic)'].apply(lambda x: x.split('/')[-1])

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [12]:
fixes = {'YEL045CD':'YEL045C','YKL118WE':'YKL118W','YOR331CF':'YOR331C','YPR099CG':'YPR099C'}
original_data['orf'] = original_data['orf'].apply(lambda x: fixes[x] if x in fixes.keys() else x)

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [No., Gene name (standard/systematic), Growth typea, Divalent cation sensitivity, Unnamed: 4, Unnamed: 5, Unnamed: 6, Pet phenotype, Quinacrineb, Ca2+content (nmole Ca2+/mg of protein), Classc, orf]
Index: []


In [15]:
data_switch = {'C':-1, 'B': -2, 'A': -3}

In [16]:
original_data['data'] = original_data['Growth typea'].apply(lambda x: data_switch[x])

In [17]:
original_data.set_index('orf', inplace=True)

In [18]:
original_data = original_data[['data']].copy()

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data.shape

(58, 1)

# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
dataset_ids = [11826]
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,11826
data_type,value
orf,
YAL016W,-1
YBR036C,-3
YBR097W,-1
YBR127C,-2
YCL007C,-2


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,11826
,data_type,value
gene_id,orf,
14,YAL016W,-1
232,YBR036C,-3
292,YBR097W,-1
321,YBR127C,-2
501,YCL007C,-2


# Normalize

In [27]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [28]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [29]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      11826           
data_type       value     valuez
gene_id orf                     
14      YAL016W    -1  -4.261380
232     YBR036C    -3 -12.784141
292     YBR097W    -1  -4.261380
321     YBR127C    -2  -8.522761
501     YCL007C    -2  -8.522761

# Print out

In [30]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [31]:
from IO.save_data_to_db3 import *

In [32]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 17351076...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  6.10it/s]


Updating the data_modified_on field...
